In [ ]:
%pylab inline
%run bizhawk.py
import subprocess
import os

In [ ]:
FPS = 60

rom_path = ''
bizhawk_path = 'BizHawk-2.2.2/'
lua_path = '../action.lua'

In [ ]:
with BizHawk(rom_path, bizhawk_path, lua_path=lua_path) as bh:
    #Speed up
    bh.send(b'client.speedmode(400);')
    
    #Advance 10 seconds
    for i in range(10 * FPS):
        bh.send(b'emu.frameadvance();')
        
    #Save and show screenshot
    code = b'client.screenshot("temp.png");'
    code += b'io.stdout:write("saved");'
    bh.send(code)
    bh.wait_until(b'saved\n')
    imshow(imread('temp.png'));
    
    #Show memory
    bh.send(b'io.stdout:write(mainmemory.getcurrentmemorydomainsize());')
    ram_size = int(bh.proc.stdout.readline()[:-1].decode('utf-8'))
    print('RAM size:', ram_size)
    
    #Show the first 8192 bytes
    ram = []
    for i in range(8192):
        bh.send(b'io.stdout:write(mainmemory.readbyte(' + str.encode(str(i)) + b'));')
        ram.append(int(bh.proc.stdout.readline()[:-1].decode('utf-8')))
    ram = np.reshape(ram, (8192 // 256, 256))
    
    figure()
    imshow(ram);
    
    #Save state to file
    bh.send(b'savestate.save("temp.state");')
    
    #Press a button on the controller
    code = b'buttons = {};'
    code += b'buttons["Start"] = 0;'
    code += b'joypad.set(buttons, 1);'
    bh.send(code)
    
    #Advance 10 more seconds
    for i in range(10 * FPS):
        bh.send(b'emu.frameadvance();')
        
    #Show another screenshot
    code = b'client.screenshot("temp.png");'
    code += b'io.stdout:write("saved");'
    bh.send(code)
    bh.wait_until(b'saved\n')
    
    figure()
    imshow(imread('temp.png'));
    
    #Load state and show screenshot to confirm load
    code = b'savestate.load("temp.state");'
    code += b'client.screenshot("temp.png");'
    code += b'io.stdout:write("saved");'
    bh.send(code)
    bh.wait_until(b'saved\n')
    
    figure()
    imshow(imread('temp.png'));
    
os.remove('temp.png')
os.remove('temp.state')